In [35]:

from bs4 import BeautifulSoup
import json
import requests

def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')

    tournaments = []

    tables = soup.find_all('table', class_='tablepress')
    
    for table in tables:
        tournament_title = table.find_previous('h2').get_text(strip=True)
        rows = table.find_all('tr')
        games = []
        for row in rows[1:]:
            cols = row.find_all('td')
            game_info = {
                'game': cols[0].get_text(strip=True),
                'time': cols[1].get_text(strip=True),
                'road_team': cols[2].get_text(strip=True),
                'road_score': cols[3].get_text(strip=True),
                'home_team': cols[4].get_text(strip=True),
                'home_score': cols[5].get_text(strip=True),
            }
            games.append(game_info)
        
        tournament = {
            'tournament': tournament_title,
            'games': games
        }
        tournaments.append(tournament)

    return tournaments



In [36]:


urlPath = 'https://d1baseball.com/college-baseball-tournament-central/'

# request the url and return it as a string

html = requests.get(urlPath).text

## Pass html to the parse_html function and return the results

tournaments = parse_html(html)

## Print the results to a text file

with open('tournaments.txt', 'w') as f:
    f.write(json.dumps(tournaments))




In [37]:
import pandas as pd
import json

# Load raw JSON data
with open('tournaments.txt') as f:
    raw_data = json.load(f)

# Create an empty list to store the cleaned data
cleaned_data = []

# Parse each tournament in raw data
for tournament in raw_data:
    tournament_name = tournament['tournament']
    games = tournament['games']
    
    # Parse each game in a tournament
    for game in games:
        # Extract information from each game
        game_info = {
            'tournament': tournament_name,
            'game': game['game'],
            'time': game['time'],
            'road_team': game['road_team'],
            'road_score': game['road_score'] if game['road_score'] != '' else None,
            'home_team': game['home_team'],
            'home_score': game['home_score'] if game['home_score'] != '' else None
        }
        
        # Append the game_info to cleaned_data list
        cleaned_data.append(game_info)

# Create a DataFrame from cleaned data
df = pd.DataFrame(cleaned_data)

# Save DataFrame to a csv file
df.to_csv('data/NCAA_D1/cleaned_tournaments.csv', index=False)


In [38]:
## look at the 'game' column and extract any text that is surrounded by parentheses into a column called 'loc_1' a delete it from the 'game' column

df['loc_1'] = df['game'].str.extract(r'\((.*?)\)')
df['game'] = df['game'].str.replace(r'\(.*?\)', '')





C:\Users\Justin\AppData\Local\Temp\ipykernel_12856\2348507559.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['game'] = df['game'].str.replace(r'\(.*?\)', '')


In [39]:
## Delete everything but numerical values from the 'game' column

df['game'] = df['game'].str.replace(r'\D', '')


C:\Users\Justin\AppData\Local\Temp\ipykernel_12856\603132715.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['game'] = df['game'].str.replace(r'\D', '')


In [40]:
## Remove the word Tournament from the 'tournament' column and rename the column to 'conference'

df['tournament'] = df['tournament'].str.replace('Tournament', '')
df.rename(columns={'tournament': 'conference'}, inplace=True)


In [41]:
# extract the date from the time column

df['date'] = df['time'].str.extract(r'(\d+/\d+)')
df['time'] = df['time'].str.replace(r'\d+/\d+', '')

# extract the time from the time column

df['time'] = df['time'].str.extract(r'(\d+:\d+)')


C:\Users\Justin\AppData\Local\Temp\ipykernel_12856\2967121956.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['time'] = df['time'].str.replace(r'\d+/\d+', '')


In [42]:
# ## Create a list of all the conference names used in the other data sets
# conf_full_names = ['America East Conference','American Athletic Conference','Atlantic 10 Conference','Atlantic Coast Conference','Atlantic Sun Conference',
#     'Big 12 Conference','Big East Conference','Big South Conference','Big Ten Conference','Big West Conference','Colonial Athletic Association','Conference USA','Horizon League','Ivy League','Metro Atlantic Athletic Conference','Mid-American Conference',
#     'Mid-Eastern Athletic Conference','Missouri Valley Conference','Mountain West Conference','Northeast Conference','Ohio Valley Conference','Pacific-12 Conference',
#     'Patriot League','Southeastern Conference','Southern Conference','Southland Conference','Southwest Athletic Conference','Sun Belt Conference','West Coast Conference',
#     'Western Athletic Conference']

df.describe()

,conference,game,time,road_team,road_score,home_team,home_score,loc_1,date
count,159,159,159,159,71,159,71,155,159
unique,29,20,26,108,18,133,18,30,10
top,ACC,1,3:00,TBA,3,Army,0,"Durham, NC",05/24
freq,12,27,21,26,11,5,9,12,71


In [43]:
# Define a dictionary mapping old conference names to their full names
conference_name_map = {
    'ACC': 'Atlantic Coast Conference',
    'America East': 'America East Conference',
    'Atlantic 10': 'Atlantic 10 Conference',
    'Atlantic Sun': 'Atlantic Sun Conference',
    'Big 12': 'Big 12 Conference',
    'Big East': 'Big East Conference',
    'Big South': 'Big South Conference',
    'Big Ten': 'Big Ten Conference',
    'Big West': 'Big West Conference',
    'Colonial': 'Colonial Athletic Association',
    'Conference USA': 'Conference USA',
    'Horizon': 'Horizon League',
    'Ivy League': 'Ivy League',
    'Metro Atlantic': 'Metro Atlantic Athletic Conference',
    'Mid-American': 'Mid-American Conference',
    'MEAC': 'Mid-Eastern Athletic Conference',
    'Missouri Valley': 'Missouri Valley Conference',
    'Mountain West': 'Mountain West Conference',
    'Northeast': 'Northeast Conference',
    'Ohio Valley': 'Ohio Valley Conference',
    'Pac-12': 'Pacific-12 Conference',
    'Patriot': 'Patriot League',
    'SEC': 'Southeastern Conference',
    'SoCon': 'Southern Conference',
    'Southland': 'Southland Conference',
    'Southwestern Athletic': 'Southwest Athletic Conference',
    'Sun Belt': 'Sun Belt Conference',
    'West Coast': 'West Coast Conference',
    'Western Athletic': 'Western Athletic Conference'
}

# First, ensure that the 'conference' column values are all lower case and stripped of leading/trailing spaces:
df['conference'] = df['conference'].str.lower().str.strip()

# Then, create a new dictionary with lower case keys:
conference_name_map_lower = {k.lower().strip(): v for k, v in conference_name_map.items()}

# Finally, replace the values in the 'conference' column using the new dictionary to create 'conference_full' column:
df['conference_full'] = df['conference'].replace(conference_name_map_lower)
df['conference_full'] = df['conference_full'].str.title()

# Drop the 'conference' column and rename the 'conference_full' column to 'conference' and reorder the columns

df.drop(columns=['conference'], inplace=True)
df.rename(columns={'conference_full': 'conference'}, inplace=True)
df = df[['date', 'time', 'conference', 'game', 'loc_1', 'road_team', 'road_score', 'home_team', 'home_score']]






In [44]:
# df.info()
df.describe()
df.head(20)


,date,time,conference,game,loc_1,road_team,road_score,home_team,home_score
0,05/23,11:00,Atlantic Coast Conference,1,"Durham, NC",Virginia Tech,7,Boston College,11
1,05/23,3:30,Atlantic Coast Conference,2,"Durham, NC",Georgia Tech,5,North Carolina,11
2,05/23,7:00,Atlantic Coast Conference,3,"Durham, NC",NC State,8,Duke,7
3,05/24,11:00,Atlantic Coast Conference,4,"Durham, NC",Pittsburgh,9,Notre Dame,5
4,05/24,3:00,Atlantic Coast Conference,5,"Durham, NC",Georgia Tech,None,Virginia,None
5,05/24,7:00,Atlantic Coast Conference,6,"Durham, NC",Virginia Tech,None,Clemson,None
6,05/25,11:00,Atlantic Coast Conference,7,"Durham, NC",Pittsburgh,None,Wake Forest,None
7,05/25,3:00,Atlantic Coast Conference,8,"Durham, NC",North Carolina,None,Virginia,None
8,05/25,7:00,Atlantic Coast Conference,9,"Durham, NC",NC State,None,Miami,None
9,05/26,11:00,Atlantic Coast Conference,10,"Durham, NC",Boston College,None,Clemson,None


In [45]:
## Save to a csv to check the data

df.to_csv('data/NCAA_D1/JSON_tournaments.csv', index=False)

## save to a json for the ncaa map

df.to_json('data/NCAA_D1/tournaments.json', orient='records')

In [46]:
df.columns

Index(['date', 'time', 'conference', 'game', 'loc_1', 'road_team',
       'road_score', 'home_team', 'home_score'],
      dtype='object')